In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
import matplotlib.pyplot as plt
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from data_loader import CustomSignalData, CustomSignalData1
from torch.autograd import Variable
from encoder import Encoder as E
from helpers import set_cmd_cb, rms_formuula, get_data, get_all_data, get_shift_data, get_operators, plot_cfs_mat, roll_data



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DEVICE = torch.device("cpu")
def getFeatureMatrix(rawDataMatrix, windowLength, windowOverlap):
    rms = lambda sig: np.sqrt(np.mean(sig**2))
    nChannels,nSamples = rawDataMatrix.shape    
    I = int(np.floor(nSamples/(windowLength-windowOverlap)))
    featMatrix = np.zeros([nChannels, I])
    for channel in range(nChannels):
        for i in range (I):
            wdwStrtIdx=i*(windowLength-windowOverlap)
            sigWin = rawDataMatrix[channel][wdwStrtIdx:(wdwStrtIdx+windowLength-1)] 
            featMatrix[channel, i] = rms(sigWin)
    featMatrixData = np.array(featMatrix)
    return featMatrixData

class FFNN(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(FFNN, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputSize, 9, bias=False),
            torch.nn.Sigmoid()
        )
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(9, outputSize, bias=False),
            # torch.nn.Softmax(dim=1)
        )

    def forward(self, x, encoder=None):
        if not encoder:
            encoder = self.encoder
        z = encoder(x)
        class_z = self.classifer(z)

        return class_z

class Operator(nn.Module):
    def __init__(self, in_features, n_rotations):
        super(Operator, self).__init__()
        """
        Args:
          in_features (int): Number of input features which should be equal to xsize.
          out_features (out): Number of output features which should be equal to ysize.
        """
        self.in_features = in_features
        self.core = torch.nn.Parameter(torch.zeros(3*self.in_features**2)- 0*torch.diag(torch.rand(3*self.in_features**2)/10))
        self.core.requires_grad = True
        self.n_rotations = n_rotations
        
    def rotate_batch(self, x, d, out_features):
      rotated = torch.empty(x.shape[0], 3*out_features*out_features, device=DEVICE)
      phies = [torch.linalg.matrix_power(self.core,i).to(DEVICE) for i in range (0,self.n_rotations+0)]
      for i in range (x.shape[0]):
        rotated[i] = phies[(d[i]+0)%4].matmul(x[i]) 
      return rotated

    def forward(self, x, d):
        """
        Args:
          x of shape (batch_size, 3, xsize, xsize): Inputs.
        
        Returns:
          y of shape (batch_size, 3*xsize^2): Outputs.
        """
        z = self.rotate_batch(x, d, self.in_features)
        return z
def get_tensor(arr):
    return torch.tensor(arr, device=DEVICE,dtype=torch.float )

def rotate_batch(x, d, out_features):
    M = torch.diag(torch.ones(8)).roll(-1,1)
    used_bases = [torch.linalg.matrix_power(M,i).to(DEVICE) for i in range (8)]
    rotated = torch.empty(x.shape, device=DEVICE)
    for i in range (x.shape[0]):
        rotated[i] = used_bases[d[i]].matmul(x[i]) 
    return rotated

def clf_acc(model, loader, masks = None, encoder = None):
    model.eval()
    correct = 0
    iter = 0
    with torch.no_grad():
        for inputs, labels,_,_ in loader:
            inputs = inputs.to(DEVICE)
            if masks is not None:
                inputs = inputs * masks[:inputs.size()[0]]
            labels = labels.to(DEVICE)
            labels = labels.flatten()
            if encoder:
                pred = model(inputs, encoder)
            else:
                pred = model(inputs)
            correct += (1-torch.abs(torch.sign(torch.argmax(pred,dim = 1)- labels))).mean().item()
            iter += 1
    return correct/iter

def compute_accuracy(a, b, loader):
    a.eval()
    b.eval()
    
    correct = 0
    iter = 0
    
    with torch.no_grad():
        for inputs1, inputs2, shift1, shift2, labels, _ in loader:
            inputs1 = inputs1.to(DEVICE)
            inputs2 = inputs2.to(DEVICE)
            shift1 = -shift1.int().flatten().to(DEVICE)
            shift2 = -shift2.int().flatten().to(DEVICE)
            labels = labels.flatten().to(DEVICE)
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            y1 = a(inputs1)
            y_tr_est1 = rotate_batch(y1,shift1,6)
            y_tr1 = b(y_tr_est1)

            y2 = a(inputs2)
            y_tr_est2 = rotate_batch(y2,shift1,6)
            y_tr2 = b(y_tr_est2)

            correct += (1-torch.abs(torch.sign(torch.argmax(y_tr1,dim = 1)- labels))).mean().item() + \
                    (1-torch.abs(torch.sign(torch.argmax(y_tr2,dim = 1)- labels))).mean().item()
            iter += 1
    return correct * 0.5 / iter

In [3]:
subject = '22'

Fs = 1000
windowLength = int(np.floor(0.1*Fs))  #160ms
windowOverlap =  int(np.floor(50/100 * windowLength))

X_train = np.zeros([0,8])
y_train= np.zeros([0])
# X_train = np.zeros([0,8])
# y_train = np.zeros([0])
for shift in range(0,1): 
    for files in sorted(os.listdir(f'Subject_{subject}/Shift_{shift}/')):
        _, class_,_, rep_ = files.split('_')
        if int(class_) in [1,2,3,4,5,6,7,8,9]:
            df = pd.read_csv(f'Subject_{subject}/Shift_{shift}/{files}',skiprows=0,sep=' ',header=None)
            data_arr = np.stack([np.array(df.T[i::8]).T.flatten().astype('float32') for i in range (8)])
            data_arr -= 121
            data_arr /= 255.0
            feaData = getFeatureMatrix(data_arr, windowLength, windowOverlap)
            
            if not class_.startswith('9'):
                rms_feature = feaData.sum(0)
                baseline = 2*rms_feature[-50:].mean()
                start_ = np.argmax(rms_feature[::1]>baseline)
                end_  = -np.argmax(rms_feature[::-1]>baseline)
                feaData = feaData.T[start_:end_]
            else:
                feaData = feaData.T

            # if rep_.startswith('2'):
            #     X_test = np.concatenate([X_test,feaData])
            #     y_test = np.concatenate([y_test,np.ones_like(feaData)[:,0]*int(class_)-1])
            
            X_train = np.concatenate([X_train,feaData])
            y_train= np.concatenate([y_train,np.ones_like(feaData)[:,0]*int(class_)-1])


In [4]:
print(X_train)
print(X_train.shape)
print(y_train)
print(y_train.shape)

[[0.00618173 0.00219444 0.00343597 ... 0.02130139 0.01734183 0.00831423]
 [0.00623178 0.0024296  0.00369729 ... 0.02960195 0.01487295 0.0117581 ]
 [0.00609314 0.00215875 0.0031776  ... 0.03120872 0.0161113  0.01154479]
 ...
 [0.00525838 0.00124636 0.00275893 ... 0.00294942 0.00233172 0.00322611]
 [0.00557388 0.0011824  0.00281467 ... 0.00294942 0.00222955 0.00307827]
 [0.00572891 0.00106079 0.00303145 ... 0.00273894 0.00193672 0.00321169]]
(4806, 8)
[0. 0. 0. ... 8. 8. 8.]
(4806,)


In [5]:
all_X_train, all_y_train, all_shift_train = get_all_data(X_train, y_train)
# all_X_test, all_y_test, all_shift_test = get_all_data(X_test, y_test)

all_X1_train, all_X2_train, all_shift_1_train, all_shift_2_train, all_y_shift_train = get_shift_data(all_X_train, all_shift_train, all_y_train)
# all_X1_test, all_X2_test, all_shift_1_test, all_shift_2_test, all_y_shift_test = get_shift_data(all_X_test, all_shift_test, all_y_test)

# Data loader
traindataset = CustomSignalData(get_tensor(X_train), get_tensor(y_train))
#testdataset = CustomSignalData(get_tensor(X_test), get_tensor(y_test))

trainloader = torch.utils.data.DataLoader(traindataset, batch_size = 1, shuffle=True)
#testloader = torch.utils.data.DataLoader(testdataset, batch_size=24, shuffle=True)

all_train_dataset = CustomSignalData(get_tensor(all_X_train), get_tensor(all_y_train))
alltrainloader = torch.utils.data.DataLoader(all_train_dataset, batch_size = 102, shuffle=True)

triplet_train_dataset = CustomSignalData1(get_tensor(all_X1_train), get_tensor(all_X2_train), get_tensor(all_shift_1_train), get_tensor(all_shift_2_train), get_tensor(all_y_shift_train))
triplettrainloader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size = 102, shuffle=True)

In [6]:
reg = LogisticRegression(penalty='l2', C=100).fit(X_train, y_train)
dump(reg, 'LogisticRegression_limit_data.joblib')

/home/eea-10/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['LogisticRegression_limit_data.joblib']

In [7]:
# Feed Forward Neural Network
inputDim = 8     # takes variable 'x' 
outputDim = 9      # takes variable 'y'
learningRate = 0.005

model = FFNN(inputDim, outputDim)
model = model.to(DEVICE)

crit = torch.nn.CrossEntropyLoss()
acc_record = []
params_clf = list(model.parameters())# + list(encoder.parameters())
optim = torch.optim.Adam(params_clf, lr=learningRate)

epochs = 200
#encoder = encoder.to(device)
for epoch in range(epochs):
    model.train()

    # Converting inputs and labels to Variable
    for inputs, labels, _, _ in alltrainloader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        labels = labels.long()
        labels = labels.flatten()
        outputs = model(inputs, None)
        optim.zero_grad()
        # get loss for the predicted output
        losss = crit(outputs, labels) #+ 0.001 * model.l1_regula()
        # get gradients w.r.t to parameters
        losss.backward()
        # update parameters
        optim.step()

torch.save(model.state_dict(), "modelwoOperator_limit_data.pt")

In [8]:
# Self-supervised learning model

encoder = E(8,8)
encoder.to(DEVICE)
classifier = FFNN(8,9)
classifier.to(DEVICE)

parameters = list(encoder.parameters()) + list(classifier.parameters())

crit1 = torch.nn.MSELoss()
crit2 = torch.nn.CrossEntropyLoss()
crit1.to(DEVICE)
crit2.to(DEVICE)
loss_record = []

optimizer = torch.optim.Adam(parameters, lr=0.002)
n_epochs = 50

for epoch in range(0,n_epochs):
    encoder.train()
    classifier.train()
    for inputs1, inputs2, shift1, shift2, labels, _ in triplettrainloader:
        inputs1 = inputs1.to(DEVICE)
        inputs2 = inputs2.to(DEVICE)
        shift1 = -shift1.int().flatten().to(DEVICE)
        shift2 = -shift2.int().flatten().to(DEVICE)
        labels = labels.long().flatten().to(DEVICE)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        y1 = encoder(inputs1)
        y_tr_est1 = rotate_batch(y1,shift1,6)
        y_tr1 = classifier(y_tr_est1)


        y2 = encoder(inputs2)
        y_tr_est2 = rotate_batch(y2,shift2,6)
        y_tr2 = classifier(y_tr_est2)

        loss =  crit1(y_tr_est1, y_tr_est2) + 0.5*crit2(y_tr1,labels)+ 0.5*crit2(y_tr2,labels)
        loss.backward()
        optimizer.step()

torch.save(classifier.state_dict(), "classifier_limit_data.pt")
torch.save(encoder.state_dict(), "encoder_limit_data.pt")
with torch.no_grad():
    encoder.eval()
    N_points = 1000
    rand_idx = np.random.choice(all_X_train.shape[0], N_points)
    y_tr = encoder(get_tensor(all_X_train[rand_idx]))
    recovered_points_ = rotate_batch(y_tr,-all_shift_train[rand_idx].flatten(), 6)
    del y_tr

torch.save(recovered_points_, "reference_points_limit_data.pt")